In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
import openpyxl

In [2]:
def get_pairing(f):
    if re.search(r'(gg01|gg18)',f):
        return '0'
    elif re.search(r'(gg04|gg11)',f):
        return '1'
    elif re.search(r'(gg06|gg19)',f):
        return '2'
    elif re.search(r'(gg08|gg20)',f):
        return '3'

In [3]:
def pick_stimuli(all_stimuli, pairing, groups):
    stimuli = []
    for stim in all_stimuli:
        if stim[9] == pairing:
            if stim[6] == '2G_SAME' and stim[8] == groups[0]:
                stimuli.append(stim)
            if stim[6] == '2G_DIFF' and stim[8] == groups[1]:
                stimuli.append(stim)
            if stim[6] == '1G' and stim[8] == groups[2]:
                stimuli.append(stim)
    return stimuli

In [4]:
def get_stimuli_for_run(all_stimuli, num_of_run, group_allocations):
    stimuli = []
    for key in group_allocations.keys():
        stimuli_pairing = pick_stimuli(all_stimuli,key,group_allocations[key][num_of_run])
        for stim in stimuli_pairing:
            stimuli.append(stim)
    return stimuli

In [5]:
def get_stimuli_for_condition(stimuli_run, condition):
    stimuli_condition = []
    for stim in stimuli_run:
        if f'{stim[6]}_{stim[7]}' == condition:
            stimuli_condition.append(stim)
    return stimuli_condition

In [6]:
def generate_possible_run(stimuli_run, conditions_run):
    for stim_pos, cond in enumerate(conditions_run):
        possible_stimuli = get_stimuli_for_condition(stimuli_run,cond)
        rand_ind = random.randrange(len(possible_stimuli))
        stimulus = possible_stimuli[rand_ind]
        conditions_run[stim_pos] = stimulus
        stimuli_run.remove(stimulus)
    
    return conditions_run

In [7]:
def get_avg_dist_group(run):
    avg_dist_group = 0
    groups = ['A','B','C']
    
    for group in groups:
        indices = [i for i, stim in enumerate(run) if stim[8] == group]
        avg_dist_group += sum(indices[cnt+1]-ind for cnt,ind in enumerate(indices[:-1]))/len(run)
        
    return avg_dist_group/len(groups)

In [8]:
def get_avg_dist_gesture(run):
    avg_dist_gesture = 0
    gestures = ['gg01', 'gg18', 'gg04', 'gg11', 'gg06', 'gg19', 'gg08', 'gg20']
    
    for gesture in gestures:
        indices = [i for i, stim in enumerate(run) if re.search(rf'{gesture}',stim[1])]
        avg_dist_gesture += sum(indices[cnt+1]-ind for cnt,ind in enumerate(indices[:-1]))/len(run)
        
    return avg_dist_gesture/len(gestures)

In [9]:
def get_avg_dist_gesture_by_actor(run):
    avg_dist_gesture_by_actor = 0
    gestures = ['gg01', 'gg18', 'gg04', 'gg11', 'gg06', 'gg19', 'gg08', 'gg20']
    actors = ['f01', 'f02','m02','p']
    gestures_by_actors = [(gesture,actor) for gesture in gestures for actor in actors]
    
    for gesture_by_actor in gestures_by_actors:
        indices = [i for i, stim in enumerate(run) if re.search(rf'{gesture_by_actor[0]}_{gesture_by_actor[1]}',stim[1])]
        avg_dist_gesture_by_actor += sum(indices[cnt+1]-ind for cnt,ind in enumerate(indices[:-1]))/len(run)

    return avg_dist_gesture_by_actor/len(gestures_by_actors)

In [10]:
def evaluate_run(run):
    avg_dist_gesture_by_actor = get_avg_dist_gesture_by_actor(run)
    avg_dist_gesture = get_avg_dist_gesture(run)
    avg_dist_group = get_avg_dist_group(run)
    
    score = 0.6*avg_dist_gesture_by_actor+0.3*avg_dist_gesture+0.1*avg_dist_group
    
    return score

In [11]:
def generate_catch_trials(dyad_stimuli, catch_pairings, catch_labels, sides, num_of_run, group_allocations):
    catch_run = []
    catch_pairings_run = catch_pairings[num_of_run*8:num_of_run*8+8]
    labels_run = catch_labels[num_of_run*8:num_of_run*8+8]
    sides_run = sides[num_of_run*8:num_of_run*8+8]
    
    for gesture_pairing, label, side in zip(catch_pairings_run, labels_run, sides_run):
        pairing_group = (get_pairing(''.join(gesture_pairing)))
        actor_group = group_allocations[pairing_group][num_of_run][label]
        actor_pairing = random.choice(actor_groups[actor_group])
        
        for stimuli in catch_stimuli:
            if re.search(rf'{side}.*{gesture_pairing[0]}_{actor_pairing[0]}.*{gesture_pairing[1]}_{actor_pairing[1]}', stimuli[1]):
                catch_run.append(stimuli)
                
    return catch_run

In [12]:
def get_best_run(dyad_stimuli, optseq_runs, num_of_run, group_allocations):
    stimuli_run = get_stimuli_for_run(dyad_stimuli, num_of_run, group_allocations)
    conditions_run = [item[1] for item in optseq_runs[num_of_run] if not(item[1] in ['NULL','Catch'])]

    best_possible_run = []
    max_score = 0

    for i in range(1,500001):
        possible_run = generate_possible_run(stimuli_run.copy(), conditions_run.copy())
        score = evaluate_run(possible_run)
        if score > max_score:
            max_score = score
            best_possible_run = possible_run
        if i % 50000 == 0:
            print(f'Score after {i} iterations: {max_score} \n')
            
    return best_possible_run

In [13]:
retval = os.getcwd()
dyad_stimuli = []
catch_stimuli = []
optseq_runs = []

for root, dirs, files in os.walk(retval):
    for f in files:
        if re.search(r'Dyads_..xlsx',f):
            df = pd.read_excel(os.path.join(root,f)) 
            stimuli_list = df[['FileName','FilePath','GestureCodeLeft','GestureCodeRight','ActorLeft','ActorRight','PairingType','InteractionType','Group']].values.tolist()
            for stimuli in stimuli_list:
                stimuli.append(get_pairing(stimuli[1]))
                dyad_stimuli.append(tuple(stimuli))
        if re.search(r'CatchTrials_.*.xlsx',f):
            df = pd.read_excel(os.path.join(root,f)) 
            catch_list = df[['FileName','FilePath','GestureCodeLeft','GestureCodeRight','ActorLeft','ActorRight','PairingType','InteractionType','Group', 'CatchSide']].values.tolist()
            for catch in catch_list:
                catch_stimuli.append(tuple(catch))
        if f.endswith('.par'):
            with open(f,'r') as f:
                run = [(float(line[:9].strip()), line[-11:].strip(), float(line[17:23])) for line in f.readlines()]
                optseq_runs.append(run)

In [14]:
PAIRINGS_2G_DIFF = [('gg01','gg18'), ('gg04','gg11'), ('gg06','gg19'), ('gg08','gg20'),
                    ('gg18','gg01'), ('gg11','gg04'), ('gg19','gg06'), ('gg20','gg08')] # get all 2G pairings
PAIRINGS_2G_SAME = [('gg01','gg01'),('gg04','gg04'),('gg06','gg06'),('gg08','gg08'),
                    ('gg11','gg11'),('gg18','gg18'),('gg19','gg19'),('gg20','gg20')]
PAIRINGS_1G = [('gg00','gg01'), ('gg00','gg04'), ('gg00','gg06'), ('gg00','gg08'), # get all 1G pairings
               ('gg00','gg11'), ('gg00','gg18'), ('gg00','gg19'), ('gg00','gg20'),
               ('gg01','gg00'), ('gg04','gg00'), ('gg06','gg00'), ('gg08','gg00'), 
               ('gg11','gg00'), ('gg18','gg00'), ('gg19','gg00'), ('gg20','gg00')]

possible_allocations = [['A','B','C'],['C','A','B'],['B','C','A']]*3

group_allocations = {}
for i in range(0,4):
    group_allocations[str(i)] = possible_allocations[0+i:6+i]
    
random.seed(61)

CATCH_PAIRINGS = PAIRINGS_1G + PAIRINGS_2G_DIFF + PAIRINGS_2G_SAME + random.sample(PAIRINGS_2G_DIFF, 4) + random.sample(PAIRINGS_2G_SAME, 4) + random.sample(PAIRINGS_1G, 8)
catch_labels = [2]*len(PAIRINGS_1G) + [1]*len(PAIRINGS_2G_DIFF) + [0]*len(PAIRINGS_2G_SAME) + [1]*4 + [0]*4 + [2]*8
sides = ['Left', 'Right']*24
actor_groups = {'A': [('f01', 'f02'),('m02','p'),('f02', 'f01'),('p','m02')],
                'B': [('f02', 'm02'),('f01','p'),('m02', 'f02'),('p','f01')],
                'C': [('f01', 'm02'),('f02','p'),('m02', 'f01'),('p','f02')]}

random.shuffle(sides)
temp = list(zip(CATCH_PAIRINGS, catch_labels))
random.shuffle(temp)
CATCH_PAIRINGS, catch_labels = zip(*temp)

In [15]:
dyad_runs = []
catch_runs = []

for num_of_run in range(0,6):
    print(f'Creating catch trials for run number {num_of_run+1}\n')
    catch_run = generate_catch_trials(dyad_stimuli, CATCH_PAIRINGS, catch_labels, sides, num_of_run, group_allocations)
    catch_runs.append(catch_run)
    print(f'Creating the sequence of stimuli for run number {num_of_run+1}\n')
    dyad_run = get_best_run(dyad_stimuli, optseq_runs, num_of_run, group_allocations)   
    dyad_runs.append(dyad_run)

Creating catch trials for run number 1

Creating the sequence of stimuli for run number 1

Score after 50000 iterations: 0.8447102864583333 

Score after 100000 iterations: 0.8447102864583333 

Score after 150000 iterations: 0.8447102864583333 

Score after 200000 iterations: 0.8447102864583333 

Score after 250000 iterations: 0.8447102864583333 

Score after 300000 iterations: 0.8449381510416667 

Score after 350000 iterations: 0.846630859375 

Score after 400000 iterations: 0.846630859375 

Score after 450000 iterations: 0.846630859375 

Score after 500000 iterations: 0.846630859375 

Creating catch trials for run number 2

Creating the sequence of stimuli for run number 2

Score after 50000 iterations: 0.8415690104166667 

Score after 100000 iterations: 0.8427083333333334 

Score after 150000 iterations: 0.8427083333333334 

Score after 200000 iterations: 0.84404296875 

Score after 250000 iterations: 0.84404296875 

Score after 300000 iterations: 0.846435546875 

Score after 350000

In [16]:
num_of_run = 1
for dyad_run, optseq_run, catch_run in zip(dyad_runs, optseq_runs, catch_runs):
    df_run = pd.DataFrame(columns=['OnsetTime','Duration','EventName','FileName','FilePath','GestureCodeLeft','GestureCodeRight','ActorLeft','ActorRight','PairingType','InteractionType','Group','CatchSide'])
    dyad_cnt, catch_cnt = 0, 0
    for event in optseq_run:
        event_info = []
        if re.search(r'NULL',event[1]):
            event_info = [event[0], event[2], event[1]]+[np.nan]*10
        elif re.search(r'Catch',event[1]):
            event_info = [event[0], event[2], event[1]]+list(catch_run[catch_cnt])
            catch_cnt += 1
        else:
            event_info = [event[0], event[2], event[1]]+list(dyad_run[dyad_cnt][:-1])+[np.nan]
            dyad_cnt += 1
        df_run.loc[len(df_run)] = event_info
    
    df_run.to_excel(f'Run-{num_of_run:03d}.xlsx')
    num_of_run += 1 